In [29]:
import pandas as pd
import numpy as np
import sklearn
#import glob
#import os
import matplotlib.pyplot as plt
from sklearn import datasets, neighbors
from mlxtend.plotting import plot_decision_regions


In [30]:
# path = r"F:\Docs\Uni-Siegen\WiSe2020-21\Fall Detection\SisFall_dataset\SisFall_dataset"
# os.chdir(path)

# nf_filenames = [i for i in glob.glob(r"F:\Docs\Uni-Siegen\WiSe2020-21\Fall Detection\SisFall_dataset\SisFall_dataset\S*\D*")]
# f_filenames = [i for i in glob.glob(r"F:\Docs\Uni-Siegen\WiSe2020-21\Fall Detection\SisFall_dataset\SisFall_dataset\S*\F*")]


# nf_combined = pd.concat([pd.read_csv(f,header=None) for f in nf_filenames ])
# nf_combined.to_csv( "nf_combined.csv", index=False, encoding='utf-8-sig')
# f_combined = pd.concat([pd.read_csv(f,header=None) for f in f_filenames ])
# f_combined.to_csv( "f_combined.csv", index=False, encoding='utf-8-sig')

In [31]:
#Step 1 Preprocessing of the Data

path1 = r"F:\Docs\Uni-Siegen\WiSe2020-21\Fall Detection\SisFall_dataset\SisFall_dataset\nf_combined.csv"
path2 = r"F:\Docs\Uni-Siegen\WiSe2020-21\Fall Detection\SisFall_dataset\SisFall_dataset\f_combined.csv"
nf_df = pd.read_csv(path1, header =0)
f_df = pd.read_csv(path2,header = 0)

In [38]:
#Converting 1st Column to float # Correcting Error in writing of csv file
nf_df["0"] = pd.to_numeric(nf_df["0"], errors='coerce')
#nf_df['0'] = nf_df['0'].astype(float)

In [39]:
nf_df.dtypes


0    float32
1    float64
2    float64
3    float64
4    float64
5    float64
6    float64
7    float64
8     object
dtype: object

In [40]:
#Remvoing NaN Values
nf_df = nf_df.dropna(axis =0) 
f_df = f_df.dropna(axis =0) 

In [41]:
#removing the data from sensor 2
nf_df = nf_df.drop(["6","7","8"], axis = 1) 
f_df = f_df.drop(["6","7","8"],axis =1) 




In [42]:
# Converting AD values
#Acceleration [g]: [(2*Range)/(2^Resolution)]*AD
#Angular velocity [°/s]: [(2*Range)/(2^Resolution)]*RD
t_acc= ((2*16)/2**13)   
r_acc= ((2*2000)/2**16)
nf_df.iloc[:,0:4] *= t_acc
nf_df.iloc[:,4:6] *= r_acc 
f_df.iloc[:,0:4] *= t_acc 
f_df.iloc[:,4:6] *= r_acc


In [43]:
# #Add the label Column
# nf_label= np.zeros((len(nf_df),1))
# nf_df = np.append(nf_df,nf_label,axis=1)
# nf_df = pd.DataFrame(nf_df[:])

# f_label= np.ones((len(f_df),1))
# f_df = np.append(f_df,f_label,axis=1)
# f_df = pd.DataFrame(f_df[:])

In [44]:
#Normalization of the Data
nf_minm = nf_df.min(axis =0)
nf_maxm = nf_df.max(axis =0)
nf_df = (nf_df-nf_minm)/(nf_maxm-nf_minm)

f_minm = f_df.min(axis =0)
f_maxm = f_df.max(axis =0)
f_df = (f_df-f_minm)/(f_maxm-f_minm)


In [45]:
#For Sckit Learning
nf_target = pd.DataFrame(np.zeros((len(nf_df),1)))
f_target = pd.DataFrame(np.ones((len(f_df),1)))

In [46]:
#Splitting Data into Training and Testing
nf_rows = np.size(nf_df,axis =0)
f_rows = np.size(f_df,axis =0)

nf_df_train = nf_df.iloc[0:int(0.8*nf_rows),:]
f_df_train = f_df.iloc[0:int(0.8*f_rows),:]
df_train = pd.concat([nf_df_train, f_df_train], axis=0, sort=False)
nf_train_target = nf_target.iloc[0:int(0.8*nf_rows),:]
f_train_target = f_target.iloc[0:int(0.8*f_rows),:]
df_train_target = pd.concat([nf_train_target, f_train_target], axis=0, sort=False)

nf_df_test = nf_df.iloc[int(0.8*nf_rows):,:]
f_df_test = f_df.iloc[int(0.8*f_rows):,:]
df_test = pd.concat([nf_df_test, f_df_test], axis=0, sort=False)
nf_test_target = nf_target.iloc[int(0.8*nf_rows):,:]
f_test_target = f_target.iloc[int(0.8*f_rows):,:]
df_test_target = pd.concat([nf_test_target, f_test_target], axis=0, sort=False)

In [47]:
np.size(nf_df_train,axis =0) + np.size(f_df_train,axis =0)

17002115

In [48]:
np.size(df_train,axis =0)

17002115

In [11]:
# classifier = KNeighborsClassifier(n_neighbors=3)
# classifier.fit(df_train, df_train_target)


D:\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=3)

In [36]:
# y_pred = classifier.predict(df_test)

In [49]:
def knn_comparison(X,Y, k):
 clf = neighbors.KNeighborsClassifier(n_neighbors=k)
 clf.fit(X, Y)
# Plotting decision region
 plot_decision_regions(X, Y, clf=clf, legend=2)
# Adding axes annotations
 plt.xlabel("Sensor Data")
 plt.ylabel("Fall or NonFall")
 plt.title("Knn with K="+ str(k))
 plt.show()


 

In [53]:

for i in [1,2,3,5]:
    knn_comparison(df_train.to_numpy(),df_train_target.to_numpy().astype(np.integer), i)
    hold(True)
    knn_comparison(df_test.to_numpy(),df_test_target.to_numpy().astype(np.integer), i)

AttributeError: 'numpy.ndarray' object has no attribute 'asarray'

In [37]:
from sklearn.metrics import r2_score
r2_score(df_test_target, y_pred)

0.9943843491793336

In [137]:
#STEP2 Model 
from collections import Counter
import math

def knn(data,query,k,distance_fn,choice_fn):
    neighbour_distances_and_indices = []
    
    #For each example in the data
    for index,example in enumerate(data):
        
        #Calculate distance between query and current exmaple from data
        distance = distance_fn(example[:-1],query)
        
        #Add distance and indices of example to ordered collection
        neighbour_distances_and_indices.append((distance,index))
        
    #Sort the ordered collection of distances and indices from smallest to largest by distance
    sorted_neighbour_distances_and_indices = sorted(neighbour_distances_and_indices)
    
    #Pick the first k entries from the sorted collection
    k_nearest_distances_and_indices = sorted_neighbour_distances_and_indices[:k]
    
    #Get the Labels of selcted k entries
    k_nearest_labels =[data[i][-1] for distance,i in k_nearest_distances_and_indices]
    
    #If regression (Choice_fn = mean),return the average of K labels
    #If Clssification (Choice_fn =mode),return mode
    return k_nearest_distances_and_indices,choice_fn(k_nearest_labels)

def mean(labels):
    return sum(labels)/len(labels)

def mode(labels):
    return Counter(labels).most_common(1)[0][0]

def euclidean_distance(point1,point2):
    sum_squared_distance = 0
    
    for i in range(len(point1)):
        sum_squared_distance += math.pow(point1[i]-point2[i],2)
        return math.sqrt(sum_squared_distance)
    
def main():
    
    #Read Data
    
    reg_k_nearest_neighbours,reg_prediction = knn(df_train, df_test,k=3,distance_fn = euclidean_distane, choice_fn = mean)
    
    if __name__ == '__main__':
        main()
    
    




In [133]:
# def euclidean_distance (row_1, row_2):
#     distance = 0.0
    
#     for i in range(len(row_1)-1):
#         distance += (row_1[i] - row_2[i])**2
#     return math.sqrt(distance)

# def get_neighbors(train, test_row, num_neighbors):
#     distances = list()
    
#     for train_row in train:
#         dist = euclidean_distance(test_row, train_row)
#         distances.append((train_row, dist))
#         # Calculating distance between testing and training data
        
#     distances.sort(key = lambda tup: tup[1])
#     neighbors = list()
#     # The list of train_row and distance tuples is sorted where a custom key is used ensuring that the second item 
#     # in the tuple (tup[1]) is used in the sorting operation.
    
#     for i in range(num_neighbors):
#         neighbors.append(distances[i][0])
#     return neighbors

# def predict_classification(train, test_row, num_neighbors):
#     neighbors = get_neighbors(train, test_row, num_neighbors)
#     output_values = [row[-1] for row in neighbors]
#     # output values taken from neighbors
#     prediction = max(set(output_values), key = output_values.count)
#     # the max() function takes a set of unique class values and calls the count on the 
#     # list of class values for each class value in the set.
#     return prediction

# #Most similar neighbors are used to make predictions

In [134]:
prediction = predict_classification(df_train, df_train[0], 3)
print('Expected %d, Got %d...' %((df_train[0][-1]), prediction))

TypeError: 'int' object is not subscriptable